In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
from functools import lru_cache
from pathlib import Path

sys.path.append(".")

base_url_node_1 = "http://204.12.168.157:8000"

from epochs import (
    EpochGroup,
    get_validator_address_from_pubkey,
    validate_epoch_group
)

GENESIS_FILE_PATH = Path("../genesis/genesis.json")

epoch_groups_chain = [EpochGroup.load(base_url_node_1, "current")]

epoch_group = epoch_groups_chain[0]
while True:
    prev_epoch_group = EpochGroup.load(base_url_node_1, epoch_group.epoch_id - 1)
    epoch_groups_chain.append(prev_epoch_group)
    epoch_group = prev_epoch_group
    if epoch_group.epoch_id == 1:
        break



In [3]:
@lru_cache(maxsize=1)
def _load_genesis_validator_addresses() -> set[str]:
    if not GENESIS_FILE_PATH.exists():
        raise FileNotFoundError(f"Genesis file not found at {GENESIS_FILE_PATH}")

    with GENESIS_FILE_PATH.open() as genesis_file:
        genesis = json.load(genesis_file)

    validator_addresses: set[str] = set()

    gen_txs = genesis.get("app_state", {}).get("genutil", {}).get("gen_txs", [])
    for tx in gen_txs:
        for message in tx.get("body", {}).get("messages", []):
            if message.get("@type") != "/cosmos.staking.v1beta1.MsgCreateValidator":
                continue

            pubkey = message.get("pubkey", {}).get("key")
            if pubkey:
                validator_addresses.add(get_validator_address_from_pubkey(pubkey))

    staking_validators = genesis.get("app_state", {}).get("staking", {}).get("validators", [])
    for validator in staking_validators:
        consensus_pubkey = validator.get("consensus_pubkey")
        if isinstance(consensus_pubkey, dict):
            pubkey = consensus_pubkey.get("key")
        else:
            pubkey = None

        if pubkey:
            validator_addresses.add(get_validator_address_from_pubkey(pubkey))

    if not validator_addresses:
        raise ValueError("No genesis validator pubkeys found in genesis file.")

    return validator_addresses


def validate_signers_total_consensus_weight(
    epoch_group_next: EpochGroup,
    epoch_group: EpochGroup,
):
    signers_addresses = epoch_group_next.created_at_block.signers_addresses
    signers_total_consensus_weight = epoch_group.signers_total_consensus_weight(signers_addresses)
    total_consensus_weight = epoch_group.get_total_consensus_weight()
    ratio = 0.0
    if total_consensus_weight:
        ratio = float(signers_total_consensus_weight) / float(total_consensus_weight)
    return ratio, total_consensus_weight, signers_total_consensus_weight


def validate_first_epoch_group(
    epoch_group: EpochGroup,
):
    genesis_validator_addresses = _load_genesis_validator_addresses()
    block_signers = set(epoch_group.created_at_block.signers_addresses)

    print(
        "Validating first epoch group: "
        f"{len(block_signers)} signers observed, {len(genesis_validator_addresses)} genesis validators expected"
    )

    missing_signers = sorted(genesis_validator_addresses - block_signers)
    unexpected_signers = sorted(block_signers - genesis_validator_addresses)

    if missing_signers:
        print("Missing genesis validator signatures:")
        for address in missing_signers:
            print(f"- {address}")

    if unexpected_signers:
        print("Unexpected block signers (not in genesis):")
        for address in unexpected_signers:
            print(f"- {address}")

    if missing_signers or unexpected_signers:
        raise AssertionError("First epoch block signatures do not match genesis validators.")

    print("First epoch block signed by all genesis validators.")
    return True
    

In [4]:
for pos, epoch_group in enumerate(epoch_groups_chain):
    if epoch_group.epoch_id == 1:
        print("Validating first epoch group")
        validate_first_epoch_group(epoch_group)
        break

    epoch_group_prev = epoch_groups_chain[pos + 1]

    validated_by_weight_portion, total_consensus_weight, signers_total_consensus_weight = validate_signers_total_consensus_weight(epoch_group, epoch_group_prev)
    print(f"Validating epoch group {epoch_group.epoch_id} from {epoch_group_prev.epoch_id}: signed by {validated_by_weight_portion} of the total weight")
    validate_epoch_group(epoch_group, prefix="  ")


Validating epoch group 69 from 68: signed by 0.8821017539588152 of the total weight
  Epoch  69 |  33 participants | 579112 | 579112
Validating epoch group 68 from 67: signed by 0.9837354147304065 of the total weight
  Epoch  68 |  28 participants | 533935 | 533521
    1 not found on chain:
      - gonkavaloper1gxnrg7jnq4364ml9d46w5et463gefl8p5eaqha |   414
Validating epoch group 67 from 66: signed by 0.9985771284068559 of the total weight
  Epoch  67 |  29 participants | 454054 | 454054
Validating epoch group 66 from 65: signed by 1.0 of the total weight
  Epoch  66 |  29 participants | 444875 | 444875
Validating epoch group 65 from 64: signed by 0.8971826481739249 of the total weight
  Epoch  65 |  29 participants | 457800 | 457800
Validating epoch group 64 from 63: signed by 0.9964258686335172 of the total weight
  Epoch  64 |  32 participants | 540969 | 540969
Validating epoch group 63 from 62: signed by 0.998253177034269 of the total weight
  Epoch  63 |  30 participants | 510054 